一度領域エンティティと、エンティティまたは値のコレクションを持てば、アプリケーションの要求を満すために、質問に答えることやデータを新しい形式に変換することができる必要がある。
命令型の言語はしばしば明示的にコレクション全体をループすることで、コレクションを処理するが、Clojureは一度にデータ集計のレベルで考え、コレクション全体へ変換を適応することを薦める。

ひとつのデータ構造を操作する関数の広範囲な組を構築するのではなく、Clojureはシーケンス抽象の変換のすべてを構築する。(??)
シーケンスはClojureのもっとも重要な部分の2つと関連するキーの抽象です。それは不変的なコレクションと変換ライブラリです。

抽象化により、われわれはただもっとも基本的なシーケンシャルな元の値を横切る様子だけを含むことを意味する。ー最初の値を取得すること、残りをシーケンスとして取得すること、そして終端をチェックするための手段。
シーケンスが実体化するにつれ、それらの値はキャッシュされ、シーケンスが実際の状態にかかわらず不変的であることが許される。(??)
このシンプルな抽象化はこれまで見てきたすべてのコレクションの変換ライブラリにあるほとんどすべての関数と接続するためには十分である。

たぶん、もっと重要なことは、シーケンス抽象の中の参加者とその頂点で操作する関数の両方がオープンなシステムであることで、この組み合わせを2つの次元において拡張可能にする。(さらなるデータと関数)(??)
任意の関数に任意のデータで接続できることは、Clojureプログラムの内外でとほうもない再利用可能性をできるようにし、Clojureプログラムをを簡潔かつ表現力豊かにするキーとなる要因でもある。

シーケンスは始めからClojureの一部でした。
Clojure 1.7で`transducer`の概念が導入されました。
そこにはさらなるシーケンシャルな処理の進化があり、入力の繰り返しの概念を部分に分割し、アプリケーションの変換であり、生成物の出力であった。(??)
それら切れ切れに分割することによって、transducerはいろいろなコンテキストをさらに広く横切るさらなるシーケンシャルな変換再利用を許した。(??)
この章の前後で、われわれはいかにシーケンスとtransducerが比較し、それぞれの最善の使い方をする方法を示すだろう。(??)

たぶん、もっとも一般的なシーケンスの変換の種類のひとつは新しいシーケンスを生成するためにシーケンスの中のすべての値に関数を適応する概念です。わたしたちはそこから始めよう。
これから見る、他の一般的な変換のいくつかに従うこと、ひとつの値に削減(reduction)、ひとつのシーケンスのある部分のフィルタリング、削除、グルーピング、ソーティング、重複の削除。
変換のパイプラインを作るためにいっしょにそれらの変換すべてを使うことを見ることで終わらせよう。(??)


# Mapping Values

データがアプリケーションの周囲を動くように、アプリケーションの一部分がそれを異なる形式で必要とすることは一般的です。
あるサブシステムは30カラムのスプレッドシートから30のキーを持つ一般的なマップにデータをインポートする。
ほかのあるサブシステムはたった5カラムだけだがひとつのエンティティに入れることを必要とし、また、さらに他のシステムは計算を実行するかそれをスクリーンに表示するために、エンティティのシーケンスからひとつのフィールドだけを必要とする。

それら全てのユースケースはある形式から別のものへのシーケンシャルなソースの中の値の変換を要求する。
Clojureにおいて、`map`関数がある関数をシーケンスの各要素へ適応してその結果の新しいシーケンスを作り出すために使われる。

例えば、画面に表示するために宇宙シュミレーションの中の各惑星エンティティの軌道周期を取り出す必要性を考えよう。
入力元は`Planet`領域エンティティのベクタです。それは、シーケンスとして扱うことができる。ー論理的には、値のリストとして。

`Planet`のシーケンシャルなコレクションを各`Planet`ごとの軌道周期のシーケンシャルなコレクションへ変換することが必要です。
この軌道周期は惑星が太陽のまわりの軌道を完全に1周する時間です。
例えば、地球では軌道周期は約365.25日です。

わたしたちは、惑星の軌道周期を計算する関数を書くことができる。
この関数の詳細を理解することは特に重要なことではない。(興味があるならば、ここに方程式を示しておく。`T`は惑星の軌道周期で、`mu(μ)`は重力定数です。)

この値は質問の惑星だけでなく、中心星の質量にも依存する。
軌道周期を計算する関数は惑星と星の質量を引数として受け取り、軌道周期を返す。

```
(defn semi-mejor-axis
  "The planet's average distance from the star"
  [p]
  (/ (+ (:aphelion p) (:perihelion p)) 2))

(defn mu [mass] (* G mass))

(defn orbital-period
  "The time it takes for a planet to make a complete
   orbit around a mass, in seconds"
   [p mass]
   (* Math/PI 2
      (Math/sqrt (/ (Math/pow (semi-major-axis p) 3)
                    (mu mass)))))
```

変換関数を持つ今、`Planet`のコレクションを軌道周期のコレクションへ変換するためにそれを使わなければならない。
Clojureの`map`関数は変換関数を惑星のベクタに適応することでシーケンシャル・ソースの中の各値を新しい値へどのようにマップするかである。

引数をひとつ(値を)取り新しい値を返す変換関数を必要とする。
`orbital-period`関数は、2つの引数の関数であるにもかかわらず、その関数を正しい形(引数ひとつ)の変換関数のなかに包まなければならない。
これはしばしば、現在の関数のスコープ内で定数値(太陽の質量)が利用可能な無名関数を使うことでおこなわれる。

```
(defn orbital-periods
  "Given a collection of planets, and a star, return the
   orbital periods of every planet."
   [planets star]
   (let [solar-mass (:mass star)]
     (map (fn [planet] (orbital-period planet solar-mass)) planets)))
```

この例のおいて、ひとつの`planet`コレクションとひとつの`star`を取り、`star`から太陽の質量を抜き出す。
そして、`planet`を取り、その`planet`と太陽の質量といっしょに`orbital-period`関数を呼び出す無名関数で`map`を呼ぶことができる。
この`map`関数はコレクションを、この関数を各`planet`へ適応しながら、返す結果をひとつのシーケンスの最後へ集めながら、わたり歩いていく。

`map`がおこなうことと、どうやって今コレクションの世界からシーケンスの世界へ交差するか、さらに深く突っ込んでゆこう。

## Sequence Processing

`map`の仕事は関数をシーケンスの中の各値へ適応することです。
Clojureがこの関数を実装する方法をシンプルにした版を見よう。
この版を`simple-map`と呼ぼう。

In [1]:
(defn simple-map
    "Map f over the elements of coll."
    [f coll]
    (when (seq coll)
        (cons (f (first coll))
              (simple-map f (rest coll)))))

#'user/simple-map

この実装はClojureのシーケンスAPIを使って書かれている。それは基本的に`seq`、`first`、`rest`、`cons`から構成されている。
この`seq`関数はコレクションが少なくとも1要素のシーケンスであるか問い掛ける。
もしそうならば、それが返り、そうでないならば、`nil`が返る。
この結果は真が偽のどちらかなので、この関数は終端のための条件判定としてしばしば使われる。

* 実際の`map`関数はこの版がここで示しているよりも、だいぶもっと複雑です。それは、本当に議論のためのものです。
  実際の`map`関数のソースをためすことは、REPLで`(source map)`を呼ぶことです。

マップされたコレクションがもっと要素を持っているとき、`cons`関数を適応します。
この`cons`関数はひとつの値と次のセルを指すポインタを含むセルを構築します。ー一連のセルは値を含んでいます。
最初のセルのなかの値はコレクションの最初の値へ変換関数`f`が適応したものとして定義されている。
残りのセルはこの関数の再帰呼び出しによって定義されており、同じ関数と入力コレクションの`rest`を通している。(??)

このシーケンスの再帰的な定義は、シーケンシャルなコレクション(リストかベクタ)へが適応されることだが、どのようなデータ構造が実装されていても、それらのどの詳細にも依存しない。(??)
このシーケンスAPIを実装するために、関係するものは、次の要素が存在するかどうかのチェックすることと、最初の要素を返すことと、残りの要素の新しいエンティティを返すことだけできなければならない。
従って、シーケンスはコレクションの論理的な見え方である。

この軌道周期の例において、どんなシーケンシャルなコレクションも他のシーケンスAPIの実装も合格でき、`map`は依然として機能するだろう。
このシーケンス抽象は一般的な`map`関数を広範囲のデータ・ソースに向かわせる。

一般的に、シーケンス関数は入力に`seqable`なものを取ることを期待する。それは`seq`が適用されるとシーケンスを産出し、また、同じものを返す。
しかしながら、この場合の結果は永続的なリストになる。元の関数に渡されたベクタのように速かったり効果的にメモリを使うということもない。
Clojureはこの特別な場合のために、特別な`mapv`関数を提供する。
`mapv`関数は`map`を使う場合と同じであるが、入力と出力がベクタとなる点が特別です。

これはほとんどのシーケンス関数の典型的な側面をハイライトする。
それらは入力(シーケンスかベクタか)の繰り返しと、(`f`関数を適用する)変換のアプリケーションと、(リストを構築するとかベクタを構築するとか)結果で何かをすることを組み合わせる。

それら3つの側面を組み合わせることはシーケンスが使われるかもしれない方法を制限する。
シーケンスの入力はひとつの抽象化であり実際なにがしかのソースで実装されるかもしれないけれども、決して再度必要とされないであろうシーケンス・ノードのキャッシュされた鎖の創造を要求する。
同様にこの関数はシーケンスの出力だけを生産する。そして、ひとつのコレクションに挿入するため、または、代りに通信チャネルと経由して値を送るために、異なる版が必要とされる。
`transducer`はこれら切れ切れに分解するために紹介される。

## Transducers

`transducer`の定義はどこから入力値がくるか、またその出力がどのように使われるか特定することを避ける代わりに、`transducer`がおこなっている実際の機能を定義する。
`map`の場合、`transducer`の仕事は関数が各要素に適用されることを保証することである。
その本質は同じで、入力の要素がコレクションかシーケンスかソケットかキューのどこからくるのかーそしてまた、出力が追加されるのはコレクションなのかファイルに保存されるのか。

われわれは`transducer`の実装について述べないだろう、なぜならそれはすこし入り組んでいるからだ。しかし、どのようにそれらが作られ、適用されるか見ることは重要である。
`map`・`transducer`を作ることは、`map`への呼び出しに入力のコレクションを省略する。

```
(defn orbital-period-transformation
  "Create a map transformation for planett->orbital-period."
  [star]
  (map #(orbital-period % (:mass star))))

```

この変換はさまざまな入力・ソースと出力条件とともに使われるだろう。
前の版の`map`と似ている出力シーケンスを生産するために、`sequence`関数とともにこの変換を使うこができる。

```
(defn orbital-periods
  [planets star]
  (sequence (orbital-period-transformation star) planets))
```

`mapv`版のように出力ベクタ作るためには、これを使おう。

```
(defn orbital-periods
  [planets star]
  (into [] (orbital-period-transformation star) planets))
```

またはリストを生産するならば

```
(defn orbital-periods
  [planets star]
  (into () (orbital-period-transformation star) planets))
```

`sequence`や`into`を使っているこの版の`orbital-periods`はどのように要素を実現化するかが異なり、`laziness`の概念と関連する。


## Laziness




# Reducing to a Value

# Filtering and Removing Values

# Take and Drop

# Sorting and Duplicate Removal

# Grouping Values

# Putting It All Together

## Selection
## Transformation
## Reduction

# Wrapping Up
